In [1]:
from code_map import data_handling, utils
import pickle


In [2]:
#return L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./one_week_collections.pkl")

In [3]:
compatible_list = utils.get_compatibility_dict(L, M)

In [4]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

Amount of markets : 62
Amount of meters : 2189
Amount of meters with direction up or both : 2040
Amount of meters with direction down or both : 2041
Amount of hours : 168


In [5]:
aggregated_results = utils.run_batched_optimization_model(L, M, H, F, Ir_hlm, Ia_hlm, Va_hm, Vp_h_m, Vm_m, R_m, R_h_l, Fu_h_l, Fd_h_l, compatible_list, "batched.log", "batched_model")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-13
Set parameter LogFile to value "batched.log"
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.1.0 23B92)Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.1.0 23B92)

CPU model: Apple M1CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threadsThread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6573432 rows, 3260208 columns and 20661508 nonzerosOptimize a model with 6573432 rows, 3260208 columns and 20661508 nonzeros
Model fingerprint: 0x7260d72bModel fingerprint: 0x7260d72b
Model has 153524 quadratic objective termsModel has 153524 quadratic objective terms
Model has 1728 quadratic constraintsModel has 1728 quadratic constraints
Variable types: 0 continuous, 3260208 integer (3260208 binary)Variable types: 0 continuous, 3260208 integer (3260208 binary)
Coeffi

: 

In [ ]:
one_week_model, x, y, w, d  = utils.run_optimization_model(L, M, H, F, Ir_hlm, Ia_hlm, Va_hm, Vp_h_m, Vm_m, R_m, R_h_l, Fu_h_l, Fd_h_l, compatible_list, "new_week_test.log", "test_model")

In [ ]:
utils.test_solution_validity(x, y, w, Va_hm, L, M, H, dominant_directions= dominant_directions, F = F)

In [ ]:
new_x_values = {(h, l, m): one_week_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}

In [ ]:
utils.compare_current_and_new_solution(current_x_pkl_file= "current_x_values_for_day.pkl", new_x_values= new_x_values, L= L, M= M, H= H, dominant_directions= dominant_directions)

In [ ]:

# Save these values
with open('current_x_values_for_week.pkl', 'wb') as f:
    pickle.dump(new_x_values, f)
                